In [1]:
!pip install pdfplumber pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 20.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1


In [8]:
import os
import uuid
import pdfplumber
import pandas as pd

# =========================
# CATEGORY & PATTERN RULES
# =========================

HOTEL_PATTERNS = [
    "hotel", "restaurant", "biryani", "mess",
    "cafe", "bakery", "dhaba", "annapoorna"
]

FUEL_PATTERNS = [
    "petrol", "diesel", "fuel", "indian oil",
    "bharat petroleum", "hp petrol"
]

TRANSPORT_PATTERNS = [
    "ola", "uber", "rapido", "redbus", "irctc"
]

SHOPPING_PATTERNS = [
    "amazon", "flipkart", "myntra", "ajio", "meesho"
]

CATEGORY_RULES = {
    "Salary": ["salary", "payroll"],
    "Rent": ["rent", "landlord", "lease"],
    "Investment": ["mutual fund", "sip", "ppf", "nps"],
    "Insurance": ["insurance", "lic", "policy"],
    "Utilities": ["electricity", "tangedco", "water", "gas", "broadband"],
    "Entertainment": ["hotstar", "netflix", "prime", "spotify"],
    "Bank Charges": ["charge", "chg", "sms alert"],
    "Interest Credit": ["interest", "int/credit"]
}


def categorize_transaction(desc: str) -> str:
    if not desc:
        return "Other"

    d = desc.lower()

    for cat, keys in CATEGORY_RULES.items():
        if any(k in d for k in keys):
            return cat

    if d.startswith("atm"):
        return "ATM Cash Withdrawal"

    if d.startswith(("neft", "imps", "rtgs")):
        return "Salary" if "salary" in d else "Transfer"

    if d.startswith("sip"):
        return "Investment"

    if d.startswith("billpay"):
        return "Utilities"

    if any(p in d for p in HOTEL_PATTERNS):
        return "Food & Dining"

    if any(p in d for p in FUEL_PATTERNS):
        return "Fuel"

    if any(p in d for p in TRANSPORT_PATTERNS):
        return "Transport"

    if any(p in d for p in SHOPPING_PATTERNS):
        return "Shopping"

    return "Other"


def extract_channel(desc: str) -> str:
    if not desc:
        return "OTHER"

    d = desc.lower()
    for c in ["upi", "pos", "atm", "neft", "imps", "ach", "sip", "billpay"]:
        if d.startswith(c):
            return c.upper()
    return "OTHER"


def extract_merchant(desc: str) -> str:
    if not desc:
        return "UNKNOWN"

    parts = desc.split("/")
    if len(parts) >= 3:
        return parts[2].strip()
    return "UNKNOWN"


# =========================
# PDF EXTRACTION UTILITIES
# =========================

PDF_DIR = "/home/shigilsasi/code/Guvi_Projects/Axis_Bank_Intelligent_Product_&_Offer_Recommendation_System_using_Machine_Learning/customer_statements"
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def clean_amount(x):
    if not x:
        return 0.0
    try:
        return float(
            x.replace(",", "")
             .replace("₹", "")
             .replace("CR", "")
             .replace("DR", "")
             .strip()
        )
    except:
        return 0.0


def parse_date(x):
    try:
        return pd.to_datetime(x, dayfirst=True, errors="coerce")
    except:
        return None


def find_column(headers, keywords):
    for h in headers:
        for k in keywords:
            if k in h:
                return h
    return None


# =========================
# MAIN EXTRACTION PIPELINE
# =========================

all_transactions = []

for file in sorted(os.listdir(PDF_DIR)):
    if not file.lower().endswith(".pdf"):
        continue

    print(f"Processing {file}")

    account_id = file.replace(".pdf", "")  # stable ID

    with pdfplumber.open(os.path.join(PDF_DIR, file)) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()

            for table in tables:
                if not table or len(table) < 2:
                    continue

                headers = [h.lower().strip() if h else "" for h in table[0]]

                date_col = find_column(headers, ["date"])
                narration_col = find_column(headers, ["narration", "description", "particular"])
                debit_col = find_column(headers, ["debit"])
                credit_col = find_column(headers, ["credit"])
                balance_col = find_column(headers, ["balance"])

                if not date_col or not balance_col:
                    continue

                for row in table[1:]:
                    rec = dict(zip(headers, row))

                    narration = rec.get(narration_col, "")

                    txn = {
                        "account_id": account_id,
                        "date": parse_date(rec.get(date_col)),
                        "narration": narration,
                        "channel": extract_channel(narration),
                        "merchant": extract_merchant(narration),
                        "category": categorize_transaction(narration),
                        "debit": clean_amount(rec.get(debit_col, "")),
                        "credit": clean_amount(rec.get(credit_col, "")),
                        "balance": clean_amount(rec.get(balance_col, "")),
                        "source_file": file
                    }

                    all_transactions.append(txn)


# =========================
# FINAL DATAFRAME
# =========================

df = pd.DataFrame(all_transactions)

if df.empty:
    raise ValueError(" No transactions extracted. Check PDF format.")

df = df.dropna(subset=["date"])
df = df.sort_values(["account_id", "date"])

output_path = os.path.join(OUTPUT_DIR, "axis_transactions.csv")
df.to_csv(output_path, index=False)

print("\n Extraction completed successfully")
print("Total transactions:", len(df))
print("Total accounts:", df["account_id"].nunique())
print("Output file:", output_path)


Processing 0001_Thiyagarajan_Subbu_Statement.pdf
Processing 0002_Aarti_Selvam_Statement.pdf
Processing 0003_Radha_Krishnan_Statement.pdf
Processing 0004_Loganathan_Surya_Statement.pdf
Processing 0005_Meena_Balaji_Statement.pdf
Processing 0006_Uma_Mohan_Statement.pdf
Processing 0007_Elango_Ganesh_Statement.pdf
Processing 0008_Aishwarya_Pandian_Statement.pdf
Processing 0009_Zafar_Surya_Statement.pdf
Processing 0010_Kavitha_Raj_Statement.pdf
Processing 0011_Thirumal_Bala_Statement.pdf
Processing 0012_Raghavan_Surya_Statement.pdf
Processing 0013_Dinesh_Arun_Statement.pdf
Processing 0014_Vasudevan_Kumar_Statement.pdf
Processing 0015_Harish_Chandra_Statement.pdf
Processing 0016_Gokul_Nathan_Statement.pdf
Processing 0017_Sanjay_Shankar_Statement.pdf
Processing 0018_Indira_Krishnan_Statement.pdf
Processing 0019_Bhuvana_Kumar_Statement.pdf
Processing 0020_Vijay_Ramesh_Statement.pdf
Processing 0021_Thamarai_Ganesh_Statement.pdf
Processing 0022_Thenmozhi_Muthu_Statement.pdf
Processing 0023_Indira

In [9]:
df.head()

,account_id,date,narration,channel,merchant,category,debit,credit,balance,source_file
0,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/973571737/LANDLORD RENT/landlord@okhdfc,UPI,LANDLORD RENT,Rent,15139.0,0.0,203044.0,0001_Thiyagarajan_Subbu_Statement.pdf
1,0001_Thiyagarajan_Subbu_Statement,2022-12-01,POS/988002/AMAZON INDIA/CARD **9369,POS,AMAZON INDIA,Shopping,3603.0,0.0,199441.0,0001_Thiyagarajan_Subbu_Statement.pdf
2,0001_Thiyagarajan_Subbu_Statement,2022-12-01,UPI/957858928/RAPIDO/rapido@ybl,UPI,RAPIDO,Transport,135.0,0.0,199306.0,0001_Thiyagarajan_Subbu_Statement.pdf
3,0001_Thiyagarajan_Subbu_Statement,2022-12-03,POS/214232/FLIPKART INDIA/CARD **9768,POS,FLIPKART INDIA,Shopping,1998.0,0.0,197308.0,0001_Thiyagarajan_Subbu_Statement.pdf
4,0001_Thiyagarajan_Subbu_Statement,2022-12-04,UPI/233699231/EATSURE/eatsure@hdfcbank,UPI,EATSURE,Other,374.0,0.0,196934.0,0001_Thiyagarajan_Subbu_Statement.pdf


In [10]:
df.isnull().sum()

account_id     0
date           0
narration      0
channel        0
merchant       0
category       0
debit          0
credit         0
balance        0
source_file    0
dtype: int64